# Multi Label Text Classification using Pytorch and 🔭 Galileo

In this tutorial, we'll train a model with PyTorch and explore the results in Galileo.

**Make sure to select GPU in your Runtime! (Runtime -> Change Runtime type)**

In [ ]:
#@title Install `dataquality` with `pip install dataquality`
try:
    import dataquality as dq
except ImportError:
    # Upgrade pip and install dependencies
    !pip install -U pip &> /dev/null
    !pip install dataquality transformers datasets torchmetrics==0.10.0 1> /dev/null

    print('👋 Installed necessary libraries and restarting runtime! This should only need to happen once.')
    print('🙏 Continue with the rest of the notebook or hit "Run All" again!')


# 1. Load Data

In [ ]:
#@title 🤗 HuggingFace Dataset
#@markdown You can find more datasets [here](https://huggingface.co/datasets?language=language:en&task_categories=task_categories:text-classification&task_ids=task_ids:multi-label-classification&sort=downloads).

dataset_name = 'go_emotions' #@param ["go_emotions", "lex_glue"] {allow-input: true}
print(f"You selected the {dataset_name} dataset")

from IPython.utils import io
from datasets import load_dataset, get_dataset_config_names

# Try to load the data. If a config (subset) is needed, pick one
try:
  with io.capture_output() as captured:
    data = load_dataset(dataset_name)
except ValueError as e:
  if "Config name is missing" not in repr(e):
    raise e

  configs = get_dataset_config_names(dataset_name)
  print(f"The dataset {dataset_name} has multiple subsets {configs}.")
  config = input(f"🖖 Enter the name of the subset to pick (or leave blank for any): ")
  if config:
    assert config in configs, f"{config} is not a valid subset"
  else:
    config = configs[0]
  with io.capture_output() as captured:
    data = load_dataset(dataset_name, name=config)


# Check that the dataset has at least train and either of validation/test
assert "train" in data and {"validation", "test"}.intersection(data), \
f"💾 The dataset {dataset_name} has either no train, or no validation or test splits, select another one."

print(f"\n🏆 Dataset {dataset_name} loaded succesfully")

# A small function for minimizing the dataset for testing
import os

def _minimize_for_ci() -> bool:
    return os.getenv("MINIMIZE_FOR_CI", "false") == "true"



In [ ]:
#@markdown Convert HF dataset to Pandas dataframes 
import pandas as pd
import numpy as np
from typing import List

def load_pandas_df(data):
  # Find the name of the ground truth column
  good_col_names = [name for name in list(data['train'].features) if "label" in name]
  if len(good_col_names) == 1:
    label_col = good_col_names[0]
  else:
    col_names = list(data['train'].features)
    print(f"The name of the columns are {col_names}.")
    label_col = input(f"🏅 Please enter the name of the column containing the labels: ")
    assert label_col in col_names, f"{label_col} is not an existing column"

  # Load the labels in a dictionary
  num_classes = len(data['train'].features[label_col].feature.names)
  labels_cols = data['train'].features[label_col].feature.int2str(range(0, num_classes))

  def binarize_label_indices(label_idxs):
    a = np.zeros(len(labels_cols), dtype=int)
    a[label_idxs] = 1
    return a

  # Load the train data into a frame
  train_data = data["train"]
  train_df = pd.DataFrame.from_dict(train_data)
  train_labels = list(map(binarize_label_indices, data['train'][label_col]))
  _train_df_labels = pd.DataFrame(train_labels, columns=labels_cols)
  train_df = pd.concat([train_df, _train_df_labels], axis=1)
  train_df['id'] = train_df.index

  # Load the test data into a frame
  test_split_name = "validation" if "validation" in data else "test"
  test_data = data[test_split_name]
  test_df = pd.DataFrame.from_dict(test_data)
  test_labels = list(map(binarize_label_indices, data[test_split_name][label_col]))
  _test_df_labels = pd.DataFrame(test_labels, columns=labels_cols)
  test_df = pd.concat([test_df, _test_df_labels], axis=1)
  test_df['id'] = test_df.index
  
  return train_df, test_df, labels_cols

# data = load_dataset(hf_dataset)
train_df, test_df, labels_cols = load_pandas_df(data)

if _minimize_for_ci():
  train_df, test_df = train_df[:10], test_df[:10]





In [ ]:
#@title 2. Preprocess dataset and prepare training
#@markdown Most PyTorch training is done with the Dataloader.
#@markdown After preprocessing the data (tokenizing) we pass it on to it.
#@markdown Galileo relies on PyTorch's DataLoader.
#@markdown
#@markdown ```python
#@markdown train_dataloader = DataLoader(train_ds, batch_size=32, shuffle=True)
#@markdown val_dataloader = DataLoader(test_ds, batch_size=BATCH_SIZE)
#@markdown ```

import torch
import torch.nn.functional as F
from torch.nn import Linear
from transformers import AutoModel
import torchmetrics
from torchmetrics import HammingDistance
from tqdm import tqdm
from transformers import AutoTokenizer
from torch.utils.data import DataLoader
BATCH_SIZE = 32

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

class MultiLabelDataset(torch.utils.data.Dataset):
    def __init__(self, dataset: pd.DataFrame, split: str, list_of_labels: List[str] = None):

        self.dataset = dataset
        self.num_samples = len(self.dataset)

        self.labels = list_of_labels
        tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")
        self.encodings = tokenizer(
            self.dataset["text"].tolist(), truncation=True, padding=True
        )
        

    def __getitem__(self, idx):
        x = torch.tensor(self.encodings["input_ids"][idx])
        attention_mask = torch.tensor(self.encodings["attention_mask"][idx])
        y = torch.tensor(self.dataset[self.labels].iloc[idx])
        return x, attention_mask, self.dataset['id'].iloc[idx], y

    def __len__(self):
        return len(self.dataset)

train_dataset = MultiLabelDataset(
    train_df, 
    split="training", 
    list_of_labels = labels_cols, 
)

test_dataset = MultiLabelDataset(
    test_df, 
    split="validation",
    list_of_labels = labels_cols,
)

train_dataloader = torch.utils.data.DataLoader(
    train_dataset, 
    batch_size=BATCH_SIZE, 
    shuffle=True,
)
val_dataloader = torch.utils.data.DataLoader(
    test_dataset, 
    batch_size=BATCH_SIZE, 
    shuffle=False,
)

class TextMultiLabelClassificationModel(torch.nn.Module):

    def __init__(self, num_tasks):
        super().__init__()
        self.feature_extractor = AutoModel.from_pretrained("distilbert-base-uncased")
        hidden_size = self.feature_extractor.config.hidden_size
        
        self.pre_classifier = torch.nn.Linear(hidden_size, hidden_size)
        self.classifier = torch.nn.Linear(hidden_size, num_tasks)

        self.dropout = torch.nn.Dropout(0.1)
        self.relu = torch.nn.ReLU()

    def forward(self, x, attention_mask):
        """Model forward function."""
        encoded_layers = self.feature_extractor(
            input_ids=x, attention_mask=attention_mask
        ).last_hidden_state
        # Extract [CLS]-token
        classification_embedding = encoded_layers[:, 0]

        emb = self.pre_classifier(classification_embedding)
        emb = self.relu(emb)
        emb = self.dropout(emb)
        logits = self.classifier(emb)

        return logits


NUM_EPOCHS = 5

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

model = TextMultiLabelClassificationModel(num_tasks=len(labels_cols))
model.to(device)

optimizer = torch.optim.AdamW(
    filter(lambda p: p.requires_grad, model.parameters()), lr=1e-5
)

train_acc = torchmetrics.Accuracy()
val_acc = torchmetrics.Accuracy()

train_hamming_distance = HammingDistance()
val_hamming_distance = HammingDistance()

loss_fn = F.binary_cross_entropy_with_logits

def extract_batch(batch):
  x, attention_mask, x_idxs, y = batch
  x = x.to(device)
  attention_mask = attention_mask.to(device)
  y = y.to(device)
  return x, y, attention_mask

def train_loop(model,dataloader,optimizer,loss_fn):
  model.train()
  running_loss = 0.0
  for batch in tqdm(dataloader):
      # print statistics
      x, y, attention_mask = extract_batch(batch)
      # zero the parameter gradients
      optimizer.zero_grad()
      # forward + backward + optimize
      logits = model(x, attention_mask)
      loss = loss_fn(logits, y.float())
      loss.backward()
      optimizer.step()
      #train_acc(logits.argmax(1), y)
      running_loss += loss.item()

def test_loop(model, dataloader, loss_fn, epoch):
  model.eval()
  validation_loss = 0.0
  pbar = tqdm(dataloader)
  for batch in pbar:
    x, y, attention_mask = extract_batch(batch)
    with torch.no_grad():
      logits = model(x, attention_mask)
      loss = loss_fn(logits, y)
    validation_loss += loss.item()
    val_hamming_distance(logits, y.float())
    pbar.set_description("[epoch %d] Validation loss: %.3f" % (epoch + 1, validation_loss))
  pbar.set_description(f"Val hamming score: {val_hamming_distance.compute()}")


## 3. Monitor with Galileo

After simply logging the orignal dataset with the following columns:
```id,text,label```

We can hook the dataquality client in our model and dataloaders.

```python
import dataquality as dq
from dataquality.integrations.torch import watch

dq.init(...)
dq.log_dataset(...)
dq.set_labels_for_run(...)
watch(model, [train_dataloader, test_dataloader])
```



In [ ]:
import dataquality as dq
from dataquality.integrations.torch import watch


# 🔭🌕 Initialize the project
dq.init('text_multi_label',
        "multi_label_text_classification_pytorch",
        f"run_{dataset_name}_04"
        )


# 🔭🌕 Galileo logging
dq.set_tasks_for_run(labels_cols, binary=True)


# 🔭🌕 Galileo logging
dq.log_data_samples(
    texts=train_df["text"],
    task_labels=train_df[labels_cols].apply(lambda row: list(row[row == 1].index.values), axis=1),
    ids=train_df["id"],
    split="train",
)

# 🔭🌕 Galileo logging
dq.log_data_samples(
    texts=test_df["text"],
    task_labels=test_df[labels_cols].apply(lambda row: list(row[row == 1].index.values), axis=1),
    ids=test_df["id"],
    split="validation",
)

# 🔭🌕 Monitor the model with Galileo
watch(model, [train_dataloader, val_dataloader])

# 4. Training

After hooking into the model we log epochs and split during the training process.
```python
dq.set_split("train")
dq.set_epoch(epoch)
```

When training is complete
```dq.finish()```
must be called

In [ ]:
from tqdm import tqdm
for epoch in range(NUM_EPOCHS):
    dq.set_epoch(epoch) # 🔭🌕 Setting the epoch
    dq.set_split("training") # 🔭🌕 Setting split to training
    train_loop(model,train_dataloader,optimizer,loss_fn)
    dq.set_split("validation") # 🔭🌕 Setting split to validation
    test_loop(model, val_dataloader, loss_fn, epoch)
print("Finished Training")

dq.finish() # 🔭🌕 Finishing the run

# General Help and Docs
- To get help with your task's requirements, call `dq.get_data_logger().doc()`
- To see more general data and model logging docs, run `dq.docs()`

In [ ]:
dq.get_data_logger().doc()
help(dq.log_dataset)

In [ ]:
# Get insights to the metrics
dq.metrics.get_dataframe(
        "example_project",
        f"run_{dataset_name}",
        "train").head()